In [1]:

# Bibliotecas
import cv2
import numpy as np
import glob
import random
import os
import matplotlib.pyplot as plt

path = os.getcwd()

# Load Yolov3
net = cv2.dnn.readNet("yolov3_training_last.weights", "yolov3_testing.cfg")


In [ ]:
# Name do objeto
classes = ["Sheep"]

# Caminho onde se encontram as imagens teste
images_path = glob.glob(path + r"\*.png")

# Pegar uma imagem aleatoria
random.shuffle(images_path)

# Estrutura da rede
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))



In [ ]:
images_path

In [ ]:
# Laço entre as imagens
for img_path in images_path:

    # Loading imagem
    img = cv2.imread(img_path)
    # Pre-processamento
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # Detectando objetos
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Informacoes do objetos
    class_ids = []    # Lista da classficacao
    confidences = []  # Lista Confianca na classificacao
    boxes = []        # Boxes delimitadores
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                # Objeto encontrado
                print(class_id)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Coordenadas do Boxe
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    print(indexes)

    # Configuracoes para print 
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 3, color, 2)


    cv2.imshow("Image", img)
    key = cv2.waitKey(0)

cv2.destroyAllWindows()